Installing the dependencies

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 9.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp39-cp39-linux_x86_64.whl size=255860 sha256=ba7388c670613b6c894770c3e23c718123796579c254ba2983dbb5ad63f65e43
  Stored in direc

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import numpy as np

In [ ]:
app = FastAPI()

In [ ]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
class model_input(BaseModel):
    
    HR: int

In [ ]:
# loading the saved model
panic_model = pickle.load(open('panicpred.sav', 'rb'))

In [ ]:
@app.post('/panic_prediction')
def panic_predd(input_parameters : model_input):
    
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)
    
    HR = input_dictionary['HR']
    hr_data = [HR,HR-5,HR+5]
  # Calculate the RR intervals from the heart rate data (in milliseconds)
    rr_data = [60000/HR for HR in hr_data]
  # Calculate the mean RR interval
    MEAN_RR = np.mean(rr_data)
  # Calculate the median RR interval
    MEDIAN_RR = np.median(rr_data)

    
    
    input_list = [HR, MEAN_RR, MEDIAN_RR]
    
    prediction = panic_model.predict([input_list])
    
    if (prediction[0] == 1):
        return 'The person is normal'
    else:
        return 'The person is in panic attack'

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [132]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://d0de-35-201-255-88.ngrok.io
INFO:     137.207.232.217:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     137.207.232.217:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [132]


##USING FLASK

In [ ]:
!pip install joblib flask pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import joblib
from flask import Flask, jsonify, request
from pyngrok import ngrok

# Load the .sav file that contains your machine learning model
model = joblib.load('/content/panicpred.sav')

# Create a Flask app
app = Flask(__name__)

# Define a route for your prediction function
@app.route('/predict', methods=['POST'])
def predict():
    # Get the HR value from the request
    hr = request.json['hr']
    
    # Convert the HR value to a 2D numpy array
    hr_array = [[hr]]
    
    # Make a prediction using your machine learning model
    prediction = model.predict(hr_array)[0]
    
    # Return the prediction as a JSON object
    return jsonify({'panic_attack': bool(prediction)})

# Run the Flask app on the ngrok URL
app.run(port=80)

# Use ngrok to create a public URL for your Flask app
public_url = ngrok.connect(port=80)
print('Public URL:', public_url)

In [ ]:
# Test your hosted model by sending a POST request to the ngrok URL with the HR value in JSON format
import requests

hr = 70
response = requests.post(public_url + '/predict', json={'hr': hr})
prediction = response.json()['panic_attack']
print('HR:', hr, '| Panic attack prediction:', prediction)